In [12]:
import sys
import csv
import os
import pandas as pd
import numpy as np
import decimal
from pyneuroml import pynml
from pyneuroml.pynml import print_comment_v
from pyneuroml.lems import LEMSSimulation
import neuroml as nml
import neuroml.writers as writers
from neuroml.nml.nml import parse as nmlparse
import random
import matplotlib
import matplotlib.pyplot as plt
random.seed(12345)

In [23]:
output_dict_nested = {}

caConc_range = [0.1, 1, 2, 3, 10]
caConc_range = [0.1]

cellSelection = ["C160802A", "C160802D", "C160802E"]
cellSelection = ["C160802A"]

for cellName in cellSelection: 
    nml_doc = nml.NeuroMLDocument(id="net")

    #Include cell file
    incl = nml.IncludeType(href=cellName+"_scaled_resample_5.cell.nml")
    nml_doc.includes.append(incl)

    #Create network
    net = nml.Network(id="net", type="networkWithTemperature", temperature="37 degC")
    nml_doc.networks.append(net)

    #Create population
    pop = nml.Population(id="pop", component=cellName, type="populationList", size="1")
    net.populations.append(pop)

    loc = nml.Location(x="0", y="0", z="0")

    inst = nml.Instance(id="0", location=loc)
    pop.instances.append(inst)
        
    delays = [300, 800]
    amplitudes = [-1, 1]
        
    for pulse_nr in range(2):
        pg = nml.PulseGenerator(id="iclamp"+str(pulse_nr), delay=str(delays[pulse_nr])+"ms", duration="10ms", 
                            amplitude=str(amplitudes[pulse_nr])+"nA")
        nml_doc.pulse_generators.append(pg)

        #Add pg to cell
        il = nml.InputList(id="clamps"+str(pulse_nr), component=pg.id, populations="pop")
        ip = nml.Input(id="0", target="../pop/0/"+cellName, segmentId="0", destination="synapses")
        il.input.append(ip)
        net.input_lists.append(il)

    nml_file = "nml_"+cellName+".net.nml"
    writers.NeuroMLWriter.write(nml_doc, nml_file)
        
    output_dict = {}
    
    #Determine output variables
    segment = 0
    target = 'net'
    sim_id = cellName
    sim_dur_ms = 1000
    dt = 0.05
    
    for caConc in caConc_range:        
        doc = nmlparse(cellName+"_scaled_resample_5.cell.nml")
        doc.cells[0].biophysical_properties.intracellular_properties.species[0].initial_concentration = str(caConc) + "mM"
        writers.NeuroMLWriter.write(doc, cellName+"_scaled_resample_5.cell.nml")

        #Create LEMS file
        ls = LEMSSimulation(sim_id, sim_dur_ms, dt, target = target)
        ls.include_neuroml2_file(nml_file)

        disp0 = 'Volts_display'
        ls.create_display(disp0, "Spiking pattern", "-90", "50")
        ls.add_line_to_display(disp0, segment, "pop/0/"+cellName+"/0/v")

        of0 = 'Volts_file'
        ls.create_output_file(of0, "%s.v.dat"%cellName)
        ls.add_column_to_output_file(of0, segment, "pop/0/"+cellName+"/0/v")
        
        disp1 = 'Gate_display'
        ls.create_display(disp1, "Gating variables", "0", "1")
        ls.add_line_to_display(disp0, segment, "pop/0/"+cellName+"/0/biophys/membraneProperties/cal_soma/cal/k/q")

        of1 = 'Gate_file'
        ls.create_output_file(of1, "%s.gates.dat"%cellName)
        ls.add_column_to_output_file(of1, segment, "pop/0/"+cellName+"/0/biophys/membraneProperties/cal_soma/cal/k/q")
        
        #Write LEMS file
        fn = "LEMS_%s.xml"%cellName
        ls.save_to_file(file_name=fn)
        out_dir, rel_filename = os.path.split(fn)
        
        #Run LEMS file with Neuron
        results_dict = pynml.run_lems_with_jneuroml_neuron(
                    rel_filename, verbose=True, nogui=True, load_saved_data=True, exec_in_dir=out_dir,
                    max_memory='400M')
        
        output_dict[str(caConc)] = results_dict
        print(results_dict.keys())    
    
    output_dict_nested[cellName] = output_dict
    df = pd.DataFrame(output_dict)
    df = pd.DataFrame.transpose(df)
    df.to_csv(cellName+".csv")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



pyNeuroML >>> Written LEMS Simulation C160802A to file: LEMS_C160802A.xml
pyNeuroML >>> Loading LEMS file: LEMS_C160802A.xml and running with jNeuroML_NEURON
pyNeuroML >>> Running jnml on LEMS_C160802A.xml with pre args: , post args:  -neuron -run -compile -nogui, in dir: , verbose: True, report: True, exit on fail: True
pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.7/site-packages/pyneuroml/lib/jNeuroML-0.10.0-jar-with-dependencies.jar"  "LEMS_C160802A.xml"  -neuron -run -compile -nogui) in directory: 
pyNeuroML >>> *** Execution of jnml has failed! ***
pyNeuroML >>> *** Command: java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.7/site-packages/pyneuroml/lib/jNeuroML-0.10.0-jar-with-dependencies.jar"  "LEMS_C160802A.xml"  -neuron -run -compile -nogui ***
pyNeuroML >>> Output: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/pyneuroml/pynml.py", line 1212, in execute_command_in_dir
 

TypeError: can only concatenate str (not "list") to str